# Purpose
1. Convert vector datasets from starting file type into GeoDataFrames.
2. Reproject all datasets into a common CRS.
3. Filter/clip all datasets to extract only records overlapping the boundary of Malheur National Forest (MNF).
4. Export each subsetted dataset to an appropriate static filetype that can be uploaded to GitHub (e.g., GeoJSON, TIF).

- USFS Forest Administrative Boundaries (https://data-usfs.hub.arcgis.com/datasets/usfs::forest-administrative-boundaries-feature-layer/explore)
- Interagency Fire Perimeter History All Years (https://data-nifc.opendata.arcgis.com/datasets/nifc::interagencyfireperimeterhistory-all-years-view/about)
- Aerial detection survey point and polygon layers from USFS (https://www.fs.usda.gov/detail/r6/forest-grasslandhealth/insects-diseases/?cid=stelprd3791643)
- USGS 1/3rd arcsecond DEMs (https://apps.nationalmap.gov/downloader/)

In [1]:
# Get folder structure paths as list. Make the paths relative. Each path needs its own variable.

In [2]:
# Get file size on disk for the datasets.

In [1]:
import geopandas as gpd
import rasterio
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import box
import json
import rioxarray as rxr
from rasterio.merge import merge
from rasterio.mask import mask
import glob
import os
import fiona
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt  # For plotting
import numpy as np  # For array manipulations (if needed)

In [2]:
# Paths
gdb_path = r"C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\downloaded_data\ads_gdb\AerialDetectionSurvey.gdb"  # Update this path
mnf_bounds = gpd.read_file(r"C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\extracted_mnf_subset\mnf_bounds.geojson")
ads_geojson_output_path = r"C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\extracted_mnf_subset\exported_ads_geojsons"

In [3]:
# List all layers (feature classes) in the geodatabase
layers = fiona.listlayers(gdb_path) # Make sure all desired layers are captured by substrings "DAMAGE_AREAS" and "DAMAGE_POINTS"
layers

['DAMAGE_POINTS_2021',
 'DAMAGE_AREAS_2021',
 'NOT_FLOWN_2021',
 'SURVEYED_AREAS_2021',
 'a_DAMAGE_AREAS_all',
 'a_DAMAGE_POINTS_all',
 'a_NOT_FLOWN_all',
 'a_SURVEYED_AREAS_all',
 'DAMAGE_AREAS_2006',
 'DAMAGE_AREAS_2008',
 'DAMAGE_AREAS_2009',
 'DAMAGE_AREAS_2004',
 'DAMAGE_AREAS_2011',
 'DAMAGE_AREAS_2012',
 'DAMAGE_AREAS_2014',
 'DAMAGE_AREAS_2015',
 'NOT_FLOWN_1947',
 'NOT_FLOWN_1948',
 'NOT_FLOWN_1949',
 'NOT_FLOWN_1973',
 'NOT_FLOWN_1974',
 'NOT_FLOWN_1975',
 'NOT_FLOWN_1976',
 'NOT_FLOWN_1977',
 'NOT_FLOWN_1972',
 'NOT_FLOWN_1979',
 'NOT_FLOWN_1980',
 'NOT_FLOWN_1981',
 'NOT_FLOWN_1982',
 'NOT_FLOWN_1983',
 'NOT_FLOWN_1984',
 'NOT_FLOWN_1978',
 'NOT_FLOWN_1985',
 'NOT_FLOWN_1986',
 'NOT_FLOWN_1987',
 'NOT_FLOWN_1988',
 'NOT_FLOWN_1989',
 'NOT_FLOWN_1990',
 'NOT_FLOWN_1991',
 'NOT_FLOWN_1992',
 'NOT_FLOWN_1993',
 'NOT_FLOWN_1950',
 'NOT_FLOWN_1952',
 'NOT_FLOWN_1953',
 'NOT_FLOWN_1955',
 'NOT_FLOWN_1956',
 'NOT_FLOWN_1958',
 'NOT_FLOWN_1959',
 'NOT_FLOWN_1961',
 'NOT_FLOWN_1962'

## Extract all layers from geodatabase

In [27]:
'''
Export all layers from geodatabase and sort into folders based on substring.
Excluded "a_DAMAGE_AREAS_all" and "a_DAMAGE_POINTS_all" as both layers are huge and cumbersome to export and analyze.
'''

# Define the mapping of substrings to folder names
folder_mapping = {
    "NOT_FLOWN": "not_flown_areas",
    "DAMAGE_AREAS": "damage_areas",
    "DAMAGE_POINTS": "damage_points",
    "SURVEYED_AREAS": "surveyed_areas"
}

layer_names = fiona.listlayers(gdb_path) # Get all layer names from the gdb
exclude_layers = {"a_DAMAGE_AREAS_all", "a_DAMAGE_POINTS_all"} # Files to exclude
target_crs = "EPSG:3857" # Define target projection (Web Mercator EPSG:3857)

# Iterate through layers and export each one
for layer_name in layer_names:
    # Read the layer as a GeoDataFrame
    gdf = gpd.read_file(gdb_path, layer=layer_name)

# Reproject to Web Mercator (EPSG:3857) if necessary
    if gdf.crs is not None and gdf.crs.to_string() != target_crs:
        gdf = gdf.to_crs(target_crs)

    # Determine the appropriate folder based on the layer name
    output_folder = None
    for key, folder in folder_mapping.items():
        if key in layer_name:
            output_folder = os.path.join(ads_geojson_output_path, folder)
            break  # Stop checking once we find a match

    # If a matching folder is found, ensure it exists
    if output_folder:
        os.makedirs(output_folder, exist_ok=True)

        # Define file path for GeoJSON
        geojson_path = os.path.join(output_folder, f"{layer_name}.geojson")

        # Export to GeoJSON
        gdf.to_file(geojson_path, driver="GeoJSON")

        print(f"Exported {layer_name} to {geojson_path} in EPSG:3857")

    except Exception as e:
        print(f"Error processing {layer_name}: {e}")

## Extract all ADS damage area polygons overlapping MNF boundary that are coded as WSB

In [5]:
# Output folder for overlapping features within MNF
output_mnf_overlap_folder = os.path.join(ads_geojson_output_path, "mnf_overlap")
os.makedirs(output_mnf_overlap_folder, exist_ok=True)

# Input folder with ADS damage area GeoJSONs
input_geojson_folder = r"C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\extracted_mnf_subset\exported_ads_geojsons\damage_areas"

def extract_mnf_overlapping_polygons(input_folder, output_folder, mnf_boundary):
    for file in os.listdir(input_folder):
        if file.endswith(".geojson"):
            file_path = os.path.join(input_folder, file)
            print(f"Checking MNF overlap for: {file}")
            
            gdf = gpd.read_file(file_path)
            
            # Ensure CRS matches
            if gdf.crs != mnf_boundary.crs:
                gdf = gdf.to_crs(mnf_boundary.crs)
            
            # Filter polygons overlapping MNF boundary
            gdf_overlap = gpd.overlay(gdf, mnf_boundary, how='intersection')
            
            # Filter for DCA_CODE == "12040"
            gdf_filtered = gdf_overlap[gdf_overlap["DCA_CODE"] == 12040]
            
            if not gdf_filtered.empty:
                output_geojson = os.path.join(output_folder, f"{os.path.splitext(file)[0]}_mnf_dca12040.geojson")
                gdf_filtered.to_file(output_geojson, driver="GeoJSON")
                print(f"Saved MNF + DCA_CODE 12040 polygons: {output_geojson}")
            else:
                print(f"No matching polygons with DCA_CODE 12040 found in {file}.")

# Run the function
extract_mnf_overlapping_polygons(input_geojson_folder, output_mnf_overlap_folder, mnf_bounds)

Checking MNF overlap for: DAMAGE_AREAS_1947.geojson
Saved MNF + DCA_CODE 12040 polygons: C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\extracted_mnf_subset\exported_ads_geojsons\mnf_overlap\DAMAGE_AREAS_1947_mnf_dca12040.geojson
Checking MNF overlap for: DAMAGE_AREAS_1948.geojson
Saved MNF + DCA_CODE 12040 polygons: C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\extracted_mnf_subset\exported_ads_geojsons\mnf_overlap\DAMAGE_AREAS_1948_mnf_dca12040.geojson
Checking MNF overlap for: DAMAGE_AREAS_1949.geojson
Saved MNF + DCA_CODE 12040 polygons: C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\extracted_mnf_subset\exported_ads_geojsons\mnf_overlap\DAMAGE_AREAS_1949_mnf_dca12040.geojson
Checking MNF overlap for: DAMAGE_AREAS_1950.geojson
Saved MNF + DCA_CODE 12040 polygons: C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\extracted_mnf_subset\exported_ads_geojsons\mnf